# Neural Machine Translation

Welcome to your first programming assignment for this week! 

You will build a Neural Machine Translation (NMT) model to translate human readable dates ("25th of June, 2009") into machine readable dates ("2009-06-25"). You will do this using an attention model, one of the most sophisticated sequence to sequence models. 

This notebook was produced together with NVIDIA's Deep Learning Institute. 

Let's load all the packages you will need for this assignment.

In [71]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

## 1 - Translating human readable dates into machine readable dates

The model you will build here could be used to translate from one language to another, such as translating from English to Hindi. However, language translation requires massive datasets and usually takes days of training on GPUs. To give you a place to experiment with these models even without using massive datasets, we will instead use a simpler "date translation" task. 

The network will input a date written in a variety of possible formats (*e.g. "the 29th of August 1958", "03/30/1968", "24 JUNE 1987"*) and translate them into standardized, machine readable dates (*e.g. "1958-08-29", "1968-03-30", "1987-06-24"*). We will have the network learn to output dates in the common machine-readable format YYYY-MM-DD. 



<!-- 
Take a look at [nmt_utils.py](./nmt_utils.py) to see all the formatting. Count and figure out how the formats work, you will need this knowledge later. !--> 

### 1.1 - Dataset

We will train the model on a dataset of 10000 human readable dates and their equivalent, standardized, machine readable dates. Let's run the following cells to load the dataset and print some examples. 

In [72]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 14879.06it/s]


In [73]:
dataset[:10]

[('18 march 1981', '1981-03-18'),
 ('wednesday december 9 2015', '2015-12-09'),
 ('monday august 13 1990', '1990-08-13'),
 ('3/4/76', '1976-03-04'),
 ('monday january 10 2005', '2005-01-10'),
 ('wednesday april 15 2015', '2015-04-15'),
 ('16 may 1998', '1998-05-16'),
 ('22 04 11', '2011-04-22'),
 ('friday july 16 1976', '1976-07-16'),
 ('wednesday january 4 1978', '1978-01-04')]

In [74]:
human_vocab

{' ': 0,
 '.': 1,
 '/': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 '<pad>': 36,
 '<unk>': 35,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'y': 34}

In [75]:
machine_vocab

{'-': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

You've loaded:
- `dataset`: a list of tuples of (human readable date, machine readable date)
- `human_vocab`: a python dictionary mapping all characters used in the human readable dates to an integer-valued index 
- `machine_vocab`: a python dictionary mapping all characters used in machine readable dates to an integer-valued index. These indices are not necessarily consistent with `human_vocab`. 
- `inv_machine_vocab`: the inverse dictionary of `machine_vocab`, mapping from indices back to characters. 

Let's preprocess the data and map the raw text data into the index values. We will also use Tx=30 (which we assume is the maximum length of the human readable date; if we get a longer input, we would have to truncate it) and Ty=10 (since "YYYY-MM-DD" is 10 characters long). 

In [76]:
inv_machine_vocab

{0: '-',
 1: '0',
 2: '1',
 3: '2',
 4: '3',
 5: '4',
 6: '5',
 7: '6',
 8: '7',
 9: '8',
 10: '9'}

In [77]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


You now have:
- `X`: a processed version of the human readable dates in the training set, where each character is replaced by an index mapped to the character via `human_vocab`. Each date is further padded to $T_x$ values with a special character (< pad >). `X.shape = (m, Tx)`
- `Y`: a processed version of the machine readable dates in the training set, where each character is replaced by the index it is mapped to in `machine_vocab`. You should have `Y.shape = (m, Ty)`. 
- `Xoh`: one-hot version of `X`, the "1" entry's index is mapped to the character thanks to `human_vocab`. `Xoh.shape = (m, Tx, len(human_vocab))`
- `Yoh`: one-hot version of `Y`, the "1" entry's index is mapped to the character thanks to `machine_vocab`. `Yoh.shape = (m, Tx, len(machine_vocab))`. Here, `len(machine_vocab) = 11` since there are 11 characters ('-' as well as 0-9). 


Lets also look at some examples of preprocessed training examples. Feel free to play with `index` in the cell below to navigate the dataset and see how source/target dates are preprocessed. 

In [78]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 18 march 1981
Target date: 1981-03-18

Source after preprocessing (indices): [ 4 11  0 24 13 28 15 20  0  4 12 11  4 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 2 10  9  2  0  1  4  0  2  9]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


## 2 - Neural machine translation with attention

If you had to translate a book's paragraph from French to English, you would not read the whole paragraph, then close the book and translate. Even during the translation process, you would read/re-read and focus on the parts of the French paragraph corresponding to the parts of the English you are writing down. 

The attention mechanism tells a Neural Machine Translation model where it should pay attention to at any step. 


### 2.1 - Attention mechanism

In this part, you will implement the attention mechanism presented in the lecture videos. Here is a figure to remind you how the model works. The diagram on the left shows the attention model. The diagram on the right shows what one "Attention" step does to calculate the attention variables $\alpha^{\langle t, t' \rangle}$, which are used to compute the context variable $context^{\langle t \rangle}$ for each timestep in the output ($t=1, \ldots, T_y$). 

<table>
<td> 
<img src="images/attn_model.png" style="width:500;height:500px;"> <br>
</td> 
<td> 
<img src="images/attn_mechanism.png" style="width:500;height:500px;"> <br>
</td> 
</table>
<caption><center> **Figure 1**: Neural machine translation with attention</center></caption>



Here are some properties of the model that you may notice: 

- There are two separate LSTMs in this model (see diagram on the left). Because the one at the bottom of the picture is a Bi-directional LSTM and comes *before* the attention mechanism, we will call it *pre-attention* Bi-LSTM. The LSTM at the top of the diagram comes *after* the attention mechanism, so we will call it the *post-attention* LSTM. The pre-attention Bi-LSTM goes through $T_x$ time steps; the post-attention LSTM goes through $T_y$ time steps. 

- The post-attention LSTM passes $s^{\langle t \rangle}, c^{\langle t \rangle}$ from one time step to the next. In the lecture videos, we were using only a basic RNN for the post-activation sequence model, so the state captured by the RNN output activations $s^{\langle t\rangle}$. But since we are using an LSTM here, the LSTM has both the output activation $s^{\langle t\rangle}$ and the hidden cell state $c^{\langle t\rangle}$. However, unlike previous text generation examples (such as Dinosaurus in week 1), in this model the post-activation LSTM at time $t$ does will not take the specific generated $y^{\langle t-1 \rangle}$ as input; it only takes $s^{\langle t\rangle}$ and $c^{\langle t\rangle}$ as input. We have designed the model this way, because (unlike language generation where adjacent characters are highly correlated) there isn't as strong a dependency between the previous character and the next character in a YYYY-MM-DD date. 

- We use $a^{\langle t \rangle} = [\overrightarrow{a}^{\langle t \rangle}; \overleftarrow{a}^{\langle t \rangle}]$ to represent the concatenation of the activations of both the forward-direction and backward-directions of the pre-attention Bi-LSTM. 

- The diagram on the right uses a `RepeatVector` node to copy $s^{\langle t-1 \rangle}$'s value $T_x$ times, and then `Concatenation` to concatenate $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$ to compute $e^{\langle t, t'}$, which is then passed through a softmax to compute $\alpha^{\langle t, t' \rangle}$. We'll explain how to use `RepeatVector` and `Concatenation` in Keras below. 

Lets implement this model. You will start by implementing two functions: `one_step_attention()` and `model()`.

**1) `one_step_attention()`**: At step $t$, given all the hidden states of the Bi-LSTM ($[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$) and the previous hidden state of the second LSTM ($s^{<t-1>}$), `one_step_attention()` will compute the attention weights ($[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$) and output the context vector (see Figure  1 (right) for details):
$$context^{<t>} = \sum_{t' = 0}^{T_x} \alpha^{<t,t'>}a^{<t'>}\tag{1}$$ 

Note that we are denoting the attention in this notebook $context^{\langle t \rangle}$. In the lecture videos, the context was denoted $c^{\langle t \rangle}$, but here we are calling it $context^{\langle t \rangle}$ to avoid confusion with the (post-attention) LSTM's internal memory cell variable, which is sometimes also denoted $c^{\langle t \rangle}$. 
  
**2) `model()`**: Implements the entire model. It first runs the input through a Bi-LSTM to get back $[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$. Then, it calls `one_step_attention()` $T_y$ times (`for` loop). At each iteration of this loop, it gives the computed context vector $c^{<t>}$ to the second LSTM, and runs the output of the LSTM through a dense layer with softmax activation to generate a prediction $\hat{y}^{<t>}$. 



**Exercise**: Implement `one_step_attention()`. The function `model()` will call the layers in `one_step_attention()` $T_y$ using a for-loop, and it is important that all $T_y$ copies have the same weights. I.e., it should not re-initiaiize the weights every time. In other words, all $T_y$ steps should have shared weights. Here's how you can implement layers with shareable weights in Keras:
1. Define the layer objects (as global variables for examples).
2. Call these objects when propagating the input.

We have defined the layers you need as global variables. Please run the following cells to create them. Please check the Keras documentation to make sure you understand what these layers are: [RepeatVector()](https://keras.io/layers/core/#repeatvector), [Concatenate()](https://keras.io/layers/merge/#concatenate), [Dense()](https://keras.io/layers/core/#dense), [Activation()](https://keras.io/layers/core/#activation), [Dot()](https://keras.io/layers/merge/#dot).

In [79]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
# densor1 = Dense(10, activation = "tanh")
densor = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

Now you can use these layers to implement `one_step_attention()`. In order to propagate a Keras tensor object X through one of these layers, use `layer(X)` (or `layer([X,Y])` if it requires multiple inputs.), e.g. `densor(X)` will propagate X through the `Dense(1)` layer defined above.

In [80]:
# GRADED FUNCTION: one_step_attention

def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a, s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
#     energies = densor2 (e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(e)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas, a])
    ### END CODE HERE ###
    
    return context

You will be able to check the expected output of `one_step_attention()` after you've coded the `model()` function.

**Exercise**: Implement `model()` as explained in figure 2 and the text above. Again, we have defined global layers that will share weights to be used in `model()`.

In [91]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

Now you can use these layers $T_y$ times in a `for` loop to generate the outputs, and their parameters will not be reinitialized. You will have to carry out the following steps: 

1. Propagate the input into a [Bidirectional](https://keras.io/layers/wrappers/#bidirectional) [LSTM](https://keras.io/layers/recurrent/#lstm)
2. Iterate for $t = 0, \dots, T_y-1$: 
    1. Call `one_step_attention()` on $[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$ and $s^{<t-1>}$ to get the context vector $context^{<t>}$.
    2. Give $context^{<t>}$ to the post-attention LSTM cell. Remember pass in the previous hidden-state $s^{\langle t-1\rangle}$ and cell-states $c^{\langle t-1\rangle}$ of this LSTM using `initial_state= [previous hidden state, previous cell state]`. Get back the new hidden state $s^{<t>}$ and the new cell state $c^{<t>}$.
    3. Apply a softmax layer to $s^{<t>}$, get the output. 
    4. Save the output by adding it to the list of outputs.

3. Create your Keras model instance, it should have three inputs ("inputs", $s^{<0>}$ and $c^{<0>}$) and output the list of "outputs".

In [92]:
# GRADED FUNCTION: model

def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(context, initial_state= [s, c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs = [X, s0,c0], outputs = outputs )
    
    ### END CODE HERE ###
    
    return model

Run the following cell to create your model.

In [93]:
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

Let's get a summary of the model to check if it matches the expected output.

In [94]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 30, 37)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 30, 64)       17920       input_6[0][0]                    
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  multiple             0           s0[0][0]                         
                                                                 lstm_10[0][0]                    
          

**Expected Output**:

Here is the summary you should see
<table>
    <tr>
        <td>
            **Total params:**
        </td>
        <td>
         185,484
        </td>
    </tr>
        <tr>
        <td>
            **Trainable params:**
        </td>
        <td>
         185,484
        </td>
    </tr>
            <tr>
        <td>
            **Non-trainable params:**
        </td>
        <td>
         0
        </td>
    </tr>
                    <tr>
        <td>
            **bidirectional_1's output shape **
        </td>
        <td>
         (None, 30, 128)  
        </td>
    </tr>
    <tr>
        <td>
            **repeat_vector_1's output shape **
        </td>
        <td>
         (None, 30, 128)  
        </td>
    </tr>
                <tr>
        <td>
            **concatenate_1's output shape **
        </td>
        <td>
         (None, 30, 256) 
        </td>
    </tr>
            <tr>
        <td>
            **attention_weights's output shape **
        </td>
        <td>
         (None, 30, 1)  
        </td>
    </tr>
        <tr>
        <td>
            **dot_1's output shape **
        </td>
        <td>
         (None, 1, 128) 
        </td>
    </tr>
           <tr>
        <td>
            **dense_2's output shape **
        </td>
        <td>
         (None, 11) 
        </td>
    </tr>
</table>


As usual, after creating your model in Keras, you need to compile it and define what loss, optimizer and metrics your are want to use. Compile your model using `categorical_crossentropy` loss, a custom [Adam](https://keras.io/optimizers/#adam) [optimizer](https://keras.io/optimizers/#usage-of-optimizers) (`learning rate = 0.005`, $\beta_1 = 0.9$, $\beta_2 = 0.999$, `decay = 0.01`)  and `['accuracy']` metrics:

In [95]:
### START CODE HERE ### (≈2 lines)
opt = model.compile(optimizer=Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01),
                    metrics=['accuracy'],
                    loss='categorical_crossentropy')
opt
### END CODE HERE ###

The last step is to define all your inputs and outputs to fit the model:
- You already have X of shape $(m = 10000, T_x = 30)$ containing the training examples.
- You need to create `s0` and `c0` to initialize your `post_activation_LSTM_cell` with 0s.
- Given the `model()` you coded, you need the "outputs" to be a list of 11 elements of shape (m, T_y). So that: `outputs[i][0], ..., outputs[i][Ty]` represent the true labels (characters) corresponding to the $i^{th}$ training example (`X[i]`). More generally, `outputs[i][j]` is the true label of the $j^{th}$ character in the $i^{th}$ training example.

In [96]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

Let's now fit the model and run it for one epoch.

In [97]:
model.fit([Xoh, s0, c0], outputs, epochs=100, batch_size=100)

Epoch 1/100


 2800/10000 [=======>......................] - ETA: 35:00 - loss: 24.0000 - dense_8_loss: 2.4000 - dense_8_acc: 0.0000e+00 - dense_8_acc_1: 0.0000e+00 - dense_8_acc_2: 0.0000e+00 - dense_8_acc_3: 0.0900 - dense_8_acc_4: 0.0200 - dense_8_acc_5: 0.0000e+00 - dense_8_acc_6: 0.0900 - dense_8_acc_7: 0.0200 - dense_8_acc_8: 0.0000e+00 - dense_8_acc_9: 0.08 - ETA: 17:26 - loss: 23.7818 - dense_8_loss: 2.4098 - dense_8_acc: 0.0000e+00 - dense_8_acc_1: 0.0000e+00 - dense_8_acc_2: 0.0000e+00 - dense_8_acc_3: 0.0450 - dense_8_acc_4: 0.5100 - dense_8_acc_5: 0.0000e+00 - dense_8_acc_6: 0.0450 - dense_8_acc_7: 0.5100 - dense_8_acc_8: 0.0000e+00 - dense_8_acc_9: 0.04 - ETA: 11:35 - loss: 23.5167 - dense_8_loss: 2.4262 - dense_8_acc: 0.0000e+00 - dense_8_acc_1: 0.0000e+00 - dense_8_acc_2: 0.0000e+00 - dense_8_acc_3: 0.0300 - dense_8_acc_4: 0.6733 - dense_8_acc_5: 0.0000e+00 - dense_8_acc_6: 0.0300 - dense_8_acc_7: 0.6733 - dense_8_acc_8: 0.0000e+00 - dense_8_acc_9: 0.02 - ETA: 8:39 - loss: 23.2155 - d

 8600/10000 [========================>.....] - ETA: 20s - loss: 19.5072 - dense_8_loss: 2.7697 - dense_8_acc: 0.2762 - dense_8_acc_1: 0.4964 - dense_8_acc_2: 0.2329 - dense_8_acc_3: 0.0510 - dense_8_acc_4: 0.9622 - dense_8_acc_5: 0.0347 - dense_8_acc_6: 0.0047 - dense_8_acc_7: 0.9067 - dense_8_acc_8: 0.1709 - dense_8_acc_9: 0.07 - ETA: 20s - loss: 19.4531 - dense_8_loss: 2.7672 - dense_8_acc: 0.2817 - dense_8_acc_1: 0.4997 - dense_8_acc_2: 0.2339 - dense_8_acc_3: 0.0507 - dense_8_acc_4: 0.9629 - dense_8_acc_5: 0.0444 - dense_8_acc_6: 0.0046 - dense_8_acc_7: 0.9083 - dense_8_acc_8: 0.1720 - dense_8_acc_9: 0.07 - ETA: 19s - loss: 19.3993 - dense_8_loss: 2.7628 - dense_8_acc: 0.2870 - dense_8_acc_1: 0.5028 - dense_8_acc_2: 0.2348 - dense_8_acc_3: 0.0507 - dense_8_acc_4: 0.9635 - dense_8_acc_5: 0.0530 - dense_8_acc_6: 0.0045 - dense_8_acc_7: 0.9098 - dense_8_acc_8: 0.1713 - dense_8_acc_9: 0.07 - ETA: 18s - loss: 19.3411 - dense_8_loss: 2.7596 - dense_8_acc: 0.2943 - dense_8_acc_1: 0.5080 -

10000/10000 [==============================] - ETA: 4s - loss: 17.3537 - dense_8_loss: 2.6403 - dense_8_acc: 0.4595 - dense_8_acc_1: 0.6143 - dense_8_acc_2: 0.2733 - dense_8_acc_3: 0.0615 - dense_8_acc_4: 0.9748 - dense_8_acc_5: 0.2611 - dense_8_acc_6: 0.0356 - dense_8_acc_7: 0.9376 - dense_8_acc_8: 0.2502 - dense_8_acc_9: 0.099 - ETA: 4s - loss: 17.2839 - dense_8_loss: 2.6365 - dense_8_acc: 0.4651 - dense_8_acc_1: 0.6180 - dense_8_acc_2: 0.2748 - dense_8_acc_3: 0.0617 - dense_8_acc_4: 0.9751 - dense_8_acc_5: 0.2667 - dense_8_acc_6: 0.0365 - dense_8_acc_7: 0.9383 - dense_8_acc_8: 0.2517 - dense_8_acc_9: 0.100 - ETA: 4s - loss: 17.2155 - dense_8_loss: 2.6326 - dense_8_acc: 0.4703 - dense_8_acc_1: 0.6213 - dense_8_acc_2: 0.2760 - dense_8_acc_3: 0.0622 - dense_8_acc_4: 0.9754 - dense_8_acc_5: 0.2718 - dense_8_acc_6: 0.0382 - dense_8_acc_7: 0.9390 - dense_8_acc_8: 0.2531 - dense_8_acc_9: 0.101 - ETA: 3s - loss: 17.1427 - dense_8_loss: 2.6282 - dense_8_acc: 0.4757 - dense_8_acc_1: 0.6251 - 

 2900/10000 [=======>......................] - ETA: 12s - loss: 10.5406 - dense_8_loss: 2.2733 - dense_8_acc: 0.9500 - dense_8_acc_1: 0.9500 - dense_8_acc_2: 0.4200 - dense_8_acc_3: 0.1300 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.7000 - dense_8_acc_6: 0.2800 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.4100 - dense_8_acc_9: 0.15 - ETA: 12s - loss: 10.3295 - dense_8_loss: 2.2568 - dense_8_acc: 0.9550 - dense_8_acc_1: 0.9550 - dense_8_acc_2: 0.4700 - dense_8_acc_3: 0.1500 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.7500 - dense_8_acc_6: 0.2150 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.4250 - dense_8_acc_9: 0.15 - ETA: 12s - loss: 10.3288 - dense_8_loss: 2.2625 - dense_8_acc: 0.9500 - dense_8_acc_1: 0.9467 - dense_8_acc_2: 0.4467 - dense_8_acc_3: 0.1467 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.7367 - dense_8_acc_6: 0.2100 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.4233 - dense_8_acc_9: 0.16 - ETA: 12s - loss: 10.3302 - dense_8_loss: 2.2471 - dense_8_acc: 0.9500 - dense_8_acc_1: 0.9450 -

...

 8700/10000 [=========================>....] - ETA: 5s - loss: 0.6275 - dense_8_loss: 0.1841 - dense_8_acc: 0.9956 - dense_8_acc_1: 0.9973 - dense_8_acc_2: 0.9944 - dense_8_acc_3: 0.9986 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9914 - dense_8_acc_6: 0.9464 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9615 - dense_8_acc_9: 0.94 - ETA: 4s - loss: 0.6276 - dense_8_loss: 0.1834 - dense_8_acc: 0.9957 - dense_8_acc_1: 0.9973 - dense_8_acc_2: 0.9945 - dense_8_acc_3: 0.9987 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9908 - dense_8_acc_6: 0.9463 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9615 - dense_8_acc_9: 0.94 - ETA: 4s - loss: 0.6339 - dense_8_loss: 0.1846 - dense_8_acc: 0.9954 - dense_8_acc_1: 0.9972 - dense_8_acc_2: 0.9944 - dense_8_acc_3: 0.9987 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9907 - dense_8_acc_6: 0.9448 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9615 - dense_8_acc_9: 0.94 - ETA: 4s - loss: 0.6309 - dense_8_loss: 0.1835 - dense_8_acc: 0.9953 - dense_8_acc_1: 0.9973 - dense_8

10000/10000 [==============================] - ETA: 1s - loss: 0.6304 - dense_8_loss: 0.1848 - dense_8_acc: 0.9952 - dense_8_acc_1: 0.9975 - dense_8_acc_2: 0.9953 - dense_8_acc_3: 0.9990 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9894 - dense_8_acc_6: 0.9469 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9623 - dense_8_acc_9: 0.93 - ETA: 1s - loss: 0.6296 - dense_8_loss: 0.1847 - dense_8_acc: 0.9953 - dense_8_acc_1: 0.9975 - dense_8_acc_2: 0.9954 - dense_8_acc_3: 0.9990 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9894 - dense_8_acc_6: 0.9470 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9625 - dense_8_acc_9: 0.93 - ETA: 1s - loss: 0.6284 - dense_8_loss: 0.1844 - dense_8_acc: 0.9953 - dense_8_acc_1: 0.9976 - dense_8_acc_2: 0.9954 - dense_8_acc_3: 0.9990 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9896 - dense_8_acc_6: 0.9470 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9628 - dense_8_acc_9: 0.93 - ETA: 1s - loss: 0.6286 - dense_8_loss: 0.1843 - dense_8_acc: 0.9954 - dense_8_acc_1: 0.9976 - dense_8

 2900/10000 [=======>......................] - ETA: 11s - loss: 0.5452 - dense_8_loss: 0.1545 - dense_8_acc: 0.9900 - dense_8_acc_1: 0.9900 - dense_8_acc_2: 1.0000 - dense_8_acc_3: 1.0000 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9900 - dense_8_acc_6: 0.9800 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9600 - dense_8_acc_9: 0.930 - ETA: 11s - loss: 0.5677 - dense_8_loss: 0.1632 - dense_8_acc: 0.9950 - dense_8_acc_1: 0.9950 - dense_8_acc_2: 1.0000 - dense_8_acc_3: 1.0000 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9950 - dense_8_acc_6: 0.9650 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9650 - dense_8_acc_9: 0.935 - ETA: 11s - loss: 0.5605 - dense_8_loss: 0.1592 - dense_8_acc: 0.9967 - dense_8_acc_1: 0.9967 - dense_8_acc_2: 1.0000 - dense_8_acc_3: 1.0000 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9900 - dense_8_acc_6: 0.9667 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9633 - dense_8_acc_9: 0.943 - ETA: 11s - loss: 0.5506 - dense_8_loss: 0.1605 - dense_8_acc: 0.9975 - dense_8_acc_1: 0.9975 - 

 8700/10000 [=========================>....] - ETA: 5s - loss: 0.6064 - dense_8_loss: 0.1781 - dense_8_acc: 0.9956 - dense_8_acc_1: 0.9985 - dense_8_acc_2: 0.9956 - dense_8_acc_3: 0.9997 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9905 - dense_8_acc_6: 0.9512 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9644 - dense_8_acc_9: 0.94 - ETA: 4s - loss: 0.6062 - dense_8_loss: 0.1778 - dense_8_acc: 0.9957 - dense_8_acc_1: 0.9985 - dense_8_acc_2: 0.9955 - dense_8_acc_3: 0.9997 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9903 - dense_8_acc_6: 0.9515 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9642 - dense_8_acc_9: 0.94 - ETA: 4s - loss: 0.6073 - dense_8_loss: 0.1787 - dense_8_acc: 0.9957 - dense_8_acc_1: 0.9985 - dense_8_acc_2: 0.9956 - dense_8_acc_3: 0.9997 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9905 - dense_8_acc_6: 0.9513 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9641 - dense_8_acc_9: 0.94 - ETA: 4s - loss: 0.6072 - dense_8_loss: 0.1790 - dense_8_acc: 0.9956 - dense_8_acc_1: 0.9984 - dense_8

10000/10000 [==============================] - ETA: 1s - loss: 0.6165 - dense_8_loss: 0.1809 - dense_8_acc: 0.9958 - dense_8_acc_1: 0.9983 - dense_8_acc_2: 0.9956 - dense_8_acc_3: 0.9989 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9897 - dense_8_acc_6: 0.9492 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9649 - dense_8_acc_9: 0.94 - ETA: 1s - loss: 0.6145 - dense_8_loss: 0.1805 - dense_8_acc: 0.9958 - dense_8_acc_1: 0.9983 - dense_8_acc_2: 0.9956 - dense_8_acc_3: 0.9989 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9898 - dense_8_acc_6: 0.9493 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9649 - dense_8_acc_9: 0.94 - ETA: 1s - loss: 0.6153 - dense_8_loss: 0.1804 - dense_8_acc: 0.9958 - dense_8_acc_1: 0.9982 - dense_8_acc_2: 0.9957 - dense_8_acc_3: 0.9989 - dense_8_acc_4: 1.0000 - dense_8_acc_5: 0.9897 - dense_8_acc_6: 0.9492 - dense_8_acc_7: 1.0000 - dense_8_acc_8: 0.9647 - dense_8_acc_9: 0.94 - ETA: 1s - loss: 0.6149 - dense_8_loss: 0.1802 - dense_8_acc: 0.9957 - dense_8_acc_1: 0.9982 - dense_8

While training you can see the loss as well as the accuracy on each of the 10 positions of the output. The table below gives you an example of what the accuracies could be if the batch had 2 examples: 

<img src="images/table.png" style="width:700;height:200px;"> <br>
<caption><center>Thus, `dense_2_acc_8: 0.89` means that you are predicting the 7th character of the output correctly 89% of the time in the current batch of data. </center></caption>


We have run this model for longer, and saved the weights. Run the next cell to load our weights. (By training a model for several minutes, you should be able to obtain a model of similar accuracy, but loading our model will save you time.) 

In [98]:
model.save_weights('model.h5')

You can now see the results on new examples.

In [99]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source = source.reshape(1, 30, 37)
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
#     print(source.shape)
#     print(Xoh.shape)
    print("source:", example)
    print("output:", ''.join(output))

source: 3 May 1979
output: 1976-02-28
source: 5 April 09
output: 1973-08-03
source: 21th of August 2016
output: 1982-11-17
source: Tue 10 Jul 2007
output: 197303-077
source: Saturday May 9 2018
output: 1980-10-15
source: March 3 2001
output: 1976-07-66
source: March 3rd 2001
output: 1976-04-26
source: 1 March 2001
output: 1976-07-17


You can also change these examples to test with your own examples. The next part will give you a better sense on what the attention mechanism is doing--i.e., what part of the input the network is paying attention to when generating a particular output character. 

## 3 - Visualizing Attention (Optional / Ungraded)

Since the problem has a fixed output length of 10, it is also possible to carry out this task using 10 different softmax units to generate the 10 characters of the output. But one advantage of the attention model is that each part of the output (say the month) knows it needs to depend only on a small part of the input (the characters in the input giving the month). We can  visualize what part of the output is looking at what part of the input.

Consider the task of translating "Saturday 9 May 2018" to "2018-05-09". If we visualize the computed $\alpha^{\langle t, t' \rangle}$ we get this: 

<img src="images/date_attention.png" style="width:600;height:300px;"> <br>
<caption><center> **Figure 8**: Full Attention Map</center></caption>

Notice how the output ignores the "Saturday" portion of the input. None of the output timesteps are paying much attention to that portion of the input. We see also that 9 has been translated as 09 and May has been correctly translated into 05, with the output paying attention to the parts of the input it needs to to make the translation. The year mostly requires it to pay attention to the input's "18" in order to generate "2018." 



### 3.1 - Getting the activations from the network

Lets now visualize the attention values in your network. We'll propagate an example through the network, then visualize the values of $\alpha^{\langle t, t' \rangle}$. 

To figure out where the attention values are located, let's start by printing a summary of the model .

In [100]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 30, 37)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 30, 64)       17920       input_6[0][0]                    
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  multiple             0           s0[0][0]                         
                                                                 lstm_10[0][0]                    
          

Navigate through the output of `model.summary()` above. You can see that the layer named `attention_weights` outputs the `alphas` of shape (m, 30, 1) before `dot_2` computes the context vector for every time step $t = 0, \ldots, T_y-1$. Lets get the activations from this layer.

The function `attention_map()` pulls out the attention values from your model and plots them.

In [101]:
attention_map = plot_attention_map(model, human_vocab, inv_machine_vocab, "Tuesday 09 Oct 1993", num = 7, n_s = 64)

InvalidArgumentError: You must feed a value for placeholder tensor 'input_4' with dtype float and shape [?,30,37]
	 [[{{node input_4}}]]

On the generated plot you can observe the values of the attention weights for each character of the predicted output. Examine this plot and check that where the network is paying attention makes sense to you.

In the date translation application, you will observe that most of the time attention helps predict the year, and hasn't much impact on predicting the day/month.

### Congratulations!


You have come to the end of this assignment 

<font color='blue'> **Here's what you should remember from this notebook**:

- Machine translation models can be used to map from one sequence to another. They are useful not just for translating human languages (like French->English) but also for tasks like date format translation. 
- An attention mechanism allows a network to focus on the most relevant parts of the input when producing a specific part of the output. 
- A network using an attention mechanism can translate from inputs of length $T_x$ to outputs of length $T_y$, where $T_x$ and $T_y$ can be different. 
- You can visualize attention weights $\alpha^{\langle t,t' \rangle}$ to see what the network is paying attention to while generating each output.

Congratulations on finishing this assignment! You are now able to implement an attention model and use it to learn complex mappings from one sequence to another. 